In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset
from torchvision import transforms
import random
import numpy as np
from PIL import Image

In [2]:
class MAEDataset(Dataset):
    def __init__(self, image_paths, patch_size=16, mask_ratio=0.75, transform=None):
        """
        Args:
            image_paths (list): List of paths to the images in the dataset.
            patch_size (int): Size of the patches to split the image into (default is 16).
            mask_ratio (float): Fraction of the patches to be masked (default is 0.75).
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.image_paths = image_paths
        self.patch_size = patch_size
        self.mask_ratio = mask_ratio
        self.transform = transform

        # Compute the number of patches in a single image
        self.img_size = 224  # Assuming image size is 224x224
        self.num_patches = (self.img_size // self.patch_size) ** 2  # Number of patches (grid)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # Load image
        image = Image.open(self.image_paths[idx]).convert('RGB')
        
        # Apply any transformations (e.g., resize, normalization)
        if self.transform:
            image = self.transform(image)
        
        # Create patches
        patches = self.image_to_patches(image)
        
        # Mask a portion of the patches
        masked_patches, mask = self.mask_patches(patches)
        
        # Reconstruct the sample
        return masked_patches, patches, mask

    def image_to_patches(self, image):
        """
        Convert an image into patches.
        Args:
            image (PIL Image or Tensor): Image to be split into patches.
        Returns:
            Tensor: Flattened patch embeddings (num_patches x patch_dim)
        """
        # Assuming image is already a tensor after transform
        patches = image.unfold(0, self.patch_size, self.patch_size).unfold(1, self.patch_size, self.patch_size)
        patches = patches.contiguous().view(-1, self.patch_size * self.patch_size * 3)  # Flatten the patches
        return patches

    def mask_patches(self, patches):
        """
        Randomly mask a portion of the patches.
        Args:
            patches (Tensor): Flattened patch embeddings (num_patches x patch_dim)
        Returns:
            Tensor: Masked patches (num_patches x patch_dim)
            Tensor: Binary mask of which patches were masked
        """
        num_patches_to_mask = int(self.num_patches * self.mask_ratio)
        mask_indices = random.sample(range(self.num_patches), num_patches_to_mask)
        
        # Create mask
        mask = torch.ones(self.num_patches)
        mask[mask_indices] = 0  # Masked patches will have a 0 in the mask

        # Mask the patches
        masked_patches = patches.clone()
        masked_patches[mask_indices] = 0  # Replace the masked patches with zeros (or any value)

        return masked_patches, mask
        